# 1 - Importar libs e carregar os DataFrames

In [1]:
from bibliotecas.data_prep import *

In [2]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# machine learning models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

# preprocessing and pipeline
from sklearn.preprocessing import (StandardScaler,  # REESCALANDO FEATURES
                                   MinMaxScaler,
                                    OneHotEncoder,
                                    OrdinalEncoder,  
                                   RobustScaler, 
                                   LabelEncoder)
from sklearn.pipeline import Pipeline # CRIANDO PIPELINES
from sklearn import set_config

import category_encoders as ce

# feature selection
from sklearn.feature_selection import SelectKBest, f_classif, chi2, RFE  # SELEÇÃO DE FEATURES
from sklearn.ensemble import RandomForestClassifier

# mode selection and metrics
from sklearn.model_selection import (train_test_split,  # METRICAS
                                     GridSearchCV,
                                     KFold,
                                     cross_val_score)
from feature_engine.encoding import CountFrequencyEncoder
from sklearn import metrics

from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             precision_score, 
                             recall_score,
                             confusion_matrix,
                             roc_auc_score,
                             roc_curve
                            )

from boruta import BorutaPy 

# warnings option
import warnings
warnings.filterwarnings('ignore')

# 2 - Lendo Arquivos

In [3]:
df = pd.read_pickle('dados/cleaned/df_nonan.pkl')
df_idh = pd.read_pickle('dados/cleaned/df_idh.pkl')
df_populacao = pd.read_excel('dados/cleaned/pop_saopaulo.xlsx')

# 3 - Associacao de Variaveis

In [4]:
target = ['obito']
cat_cols = ['cs_sexo','asma','cardiopatia','diabetes','doenca_hematologica','doenca_hepatica','doenca_neurologica','doenca_renal','imunodepressao','obesidade','outros_fatores_de_risco','pneumopatia','puerpera','sindrome_de_down']
num_cols = ['idade']

## 3.1 - Variaveis numéricas

In [5]:
df[num_cols+target].corr()['obito']

idade    0.266945
obito    1.000000
Name: obito, dtype: float64

## 3.2 - Variaveis Cetegoricas

In [6]:
# V de Cramer
cramer_v(df, cat_cols+target)['obito']

cs_sexo                    0.037105
asma                       0.382664
cardiopatia                0.331630
diabetes                   0.354265
doenca_hematologica        0.383208
doenca_hepatica            0.383688
doenca_neurologica         0.395646
doenca_renal               0.382196
imunodepressao             0.375315
obesidade                  0.391230
outros_fatores_de_risco    0.421101
pneumopatia                0.392001
puerpera                   0.381258
sindrome_de_down           0.382270
obito                      0.999997
Name: obito, dtype: float64

In [7]:
# substituindo valores ignoradoe e aplicando o V de Cramer
df1 = df.replace(['IGNORADO'],['NÃO'])
cramer_v(df1, cat_cols+target)['obito']

cs_sexo                    0.037105
asma                       0.066116
cardiopatia                0.233758
diabetes                   0.210347
doenca_hematologica        0.058346
doenca_hepatica            0.072518
doenca_neurologica         0.163480
doenca_renal               0.123321
imunodepressao             0.059066
obesidade                  0.176974
outros_fatores_de_risco    0.310317
pneumopatia                0.141470
puerpera                   0.014077
sindrome_de_down           0.029828
obito                      0.999997
Name: obito, dtype: float64

# 4 - Feature Engineering

Como vimos que diferentes faixas de idade possuem mais ou menos chance de sobreviver, vamos criar uma variável categórica com a faixa de idade do paciente

In [8]:
# df['faixa_etaria'] , age_bins = pd.cut(df['idade'], 5, retbins=True)


# Alterando os limites das bandas para começar em 0 e terminar em 110
# age_bins[0] = 0
# age_bins[-1] = 110

Criaremos dummys para facilitar a análise

In [9]:
# Convertendo todas as colunas categoricas para dummy's
resposta = {'NÃO':0,'SIM':1,'MASCULINO':0,'FEMININO':1}
df1 = df1.replace(resposta)
df1.head(4)

,nome_munic,codigo_ibge,idade,cs_sexo,data_inicio_sintomas,obito,asma,cardiopatia,diabetes,doenca_hematologica,doenca_hepatica,doenca_neurologica,doenca_renal,imunodepressao,obesidade,outros_fatores_de_risco,pneumopatia,puerpera,sindrome_de_down
0,sao paulo,3550308,45.0,0,2021-03-18T00:00:00Z,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,sao paulo,3550308,39.0,0,2021-07-07T00:00:00Z,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,sao paulo,3550308,59.0,0,2021-01-20T00:00:00Z,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,maua,3529401,27.0,0,2021-02-20T00:00:00Z,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# 5 - Feature Selection

De todas as Features que temos, vamos identificar quais, de fato, são relevates para o nosso problema. Faremos isso em duas etapas.


## 5.1 - Filter Method

Nesta etapa verificamos se as relações observadas entre as variáveis e o target são estatisticamente significativas. Para isso, utilizamos testes de hipótese como ANOVA e chi2. Analisando os p-valores, identificamos variáveis que não são estatisticamente significativas e as removemos do conjunto.

In [10]:
colunms_drop = ['data_inicio_sintomas','nome_munic','codigo_ibge','obito']

In [11]:
# variaveis explicativas
X = df1.drop(columns=colunms_drop)
# variável target
y = df1['obito']

In [12]:
seed = 10
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.3,random_state = seed)

#### Variaveis Numéricas

In [13]:
# Seleciona apenas colunas numericas do treino
X_train_num = X_train[num_cols]

In [14]:
# Faz o teste de hipotese utilizando ANOVA
anova_result = f_classif(X_train_num, y_train)

In [15]:
# Organiza os p-valores em uma série
p_values_num_features = pd.Series(anova_result[1])
p_values_num_features.index = num_cols

In [16]:
# Filtra apenas colunas que tiveram um p-valor inferior a 0.05
filter_num_features = p_values_num_features[p_values_num_features < 0.05].index.tolist()
filter_num_features

['idade']

#### Variáveis Categóricas

Para não precisar fazer o encoding das variáveis categóricas agora, vamos utilizar o teste de chi2 do scipy

In [17]:
# Faz o teste chi2 para cada uma das variáveis
p_values_cat_features = {}
for col in cat_cols:
    # Cria tabela de contingencia
    df_cross = pd.crosstab(df1[col], df1['obito'])
    # Aplica o teste e extrai o p-valor
    p_value = stats.chi2_contingency(df_cross)[1]
    # Armazena coluna e p-valor em um dict
    p_values_cat_features[col] = p_value

In [18]:
# Organiza p-valores e colunas em uma série
p_values_cat_features = pd.Series(p_values_cat_features)

In [19]:
# Filtra colunas com p-valor inferior a 0.05
filter_cat_features = p_values_cat_features[p_values_cat_features < 0.05].index.tolist()
filter_cat_features

['cs_sexo',
 'asma',
 'cardiopatia',
 'diabetes',
 'doenca_hematologica',
 'doenca_hepatica',
 'doenca_neurologica',
 'doenca_renal',
 'imunodepressao',
 'obesidade',
 'outros_fatores_de_risco',
 'pneumopatia',
 'puerpera',
 'sindrome_de_down']

## 5.2 - Wrapper Method

Aqui utilizamos alguns algoritmos como Boruta ou RFE, cuja ideia é identificar quais as variáveis que, de fato contribuem para o modelo. Tais algoritmos fazem isso, por exemplo, testando a feature contra uma versão randomizada de si mesma ou avaliando se um modelo com a feature performa signficativamente melhor do que o mesmo modelo sem a feature em questão.


### 5.2.1 - Frequency Encoder

Para utilizar os Wrapper Methods precisamos fazer encoding das variáveis categóricas. Para isso, vamos utilizar o Frequency Encoder

In [20]:
# Primeiro precisamos passar todas as variáveis categóricas para object
for col in cat_cols:
    X_train[col] = X_train[col].astype('object')

In [21]:
# Inicializa o encoder
fenc = CountFrequencyEncoder(encoding_method='frequency', variables=cat_cols)
# Aplica a transformacao em X_train
X_train_encoded = fenc.fit_transform(X_train)

In [22]:
# Iniacializa algoritmo Random Forest para utilizacao no Boruta
forest = RandomForestClassifier(n_jobs=-1, max_depth=5)
# Inicializa o Boruta
boruta_selector = BorutaPy(forest, n_estimators=50, max_iter=100, random_state=42)

In [23]:
# Ajusta o Boruta sobre os dados de treino
boruta_selector.fit(np.array(X_train_encoded), y_train)

BorutaPy(estimator=RandomForestClassifier(max_depth=5, n_estimators=50,
                                          n_jobs=-1,
                                          random_state=RandomState(MT19937) at 0x16BC425CF40),
         n_estimators=50, random_state=RandomState(MT19937) at 0x16BC425CF40)

In [24]:
# Visualiza variáveis significativas
boruta_selector.support_

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True])

In [25]:
# Visualiza variáveis com significancia duvidosa
boruta_selector.support_weak_

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False])

In [26]:
# Coleta nome das variáveis consideradas significativas pelo Boruta
features_boruta = X_train_encoded.loc[:, boruta_selector.support_].columns.tolist()

In [27]:
# Juntando variáveis selecionadas pelo filtro
features_filter = filter_num_features + filter_cat_features

### 5.2.1 - RFE

In [28]:
#Aplicando o RFE
model_logistic = LogisticRegression()
#quero que selecione as 5 features mais importantes
rfe = RFE(model_logistic, n_features_to_select = 5, step = 1) 
# modelo utilizado de regressão logistica /  n_features_to_select =quantidade de features a permanecer / step = de quanto em quantas colunas eu removo 
fit = rfe.fit(X, y)

In [29]:
# visualizando os índices das colunas selecionadas
cols = fit.get_support(indices=True)

# Filtrando o dataset com apenas as colunas selecionadas
features_rfe = X.iloc[:,cols].columns.tolist()

## 5.3 - Resultados

In [30]:
# Define variáveis selecionadas como variáveis em comum entre boruta e filtro
selected_features = list(set(features_boruta).intersection(set(features_filter)).intersection(set(features_rfe)))

In [31]:
# Adiciona às variáveis selecionadas a Faixa de Idade, o tamanho da família e o local de embarque
selected_features.extend(['idade', 'cs_sexo'])

In [32]:
selected_features

['outros_fatores_de_risco',
 'doenca_neurologica',
 'cardiopatia',
 'pneumopatia',
 'doenca_hepatica',
 'idade',
 'cs_sexo']

In [33]:
# Filtra X_train para variáveis selecionadas
X_train = X_train[selected_features]

In [34]:
X_train.head(4)

,outros_fatores_de_risco,doenca_neurologica,cardiopatia,pneumopatia,doenca_hepatica,idade,cs_sexo
3819896,0,0,1,0,0,73.0,0
1705389,0,0,0,0,0,61.0,0
1378857,0,0,0,0,0,64.0,0
2872153,1,0,1,0,0,68.0,1


In [35]:
# Atualiza variáveis categóricas e numéricas
cat_cols_updated = ['cardiopatia','diabetes', 'doenca_neurologica', 'obesidade', 'outros_fatores_de_risco','cs_sexo']
num_cols_updated = ['idade']

In [36]:
new_selected_features = cat_cols_updated + num_cols_updated

# 6 - Exportando o dataframe apenas com as variaveis selecionadas

In [37]:
df1[new_selected_features + target + ['data_inicio_sintomas']].to_pickle('dados\cleaned\df_preped.pkl')